In [2]:
import csv
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB, Model

In [3]:
with open('volume_data.csv', 'r') as fin:
    volume = int(list(csv.reader(fin))[1][1])

In [11]:
with open('demand_data.csv', 'r') as fin:
    temp = list(csv.reader(fin))
    d = pd.DataFrame(temp[1:])
    d.columns = temp[0]
    d.drop('Month', axis = 1, inplace = True)
    t = d.shape[0]
    i = d.shape[1]
    d = d.to_numpy().astype(int)
i

50

In [5]:
with open('product_data.csv', 'r') as fin:
    temp = list(csv.reader(fin))
    product = pd.DataFrame(temp[1:])
    product.columns = temp[0]
    product.drop('Product', axis = 1, inplace = True)
    v = product.iloc[:, 0].to_numpy().astype(float)
    s = product.iloc[:, 0].to_numpy().astype(float)

In [6]:
with open('cost_data.csv', 'r') as fin:
    temp = list(csv.reader(fin))
    c = pd.DataFrame(temp[1:])
    c.columns = temp[0]
    c.drop('Month', axis = 1, inplace = True)
    c = c.to_numpy().astype(float)

In [7]:
m = Model('Assignment 2')
m.setParam('OutputFlag', True)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-06


In [8]:
x = m.addVars(t, i, name = 'x')
y = m.addVars(t, i, name = 'y')
x

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1): <gurobi.Var *Awaiting Model Update*>,
 (0, 2): <gurobi.Var *Awaiting Model Update*>,
 (0, 3): <gurobi.Var *Awaiting Model Update*>,
 (0, 4): <gurobi.Var *Awaiting Model Update*>,
 (0, 5): <gurobi.Var *Awaiting Model Update*>,
 (0, 6): <gurobi.Var *Awaiting Model Update*>,
 (0, 7): <gurobi.Var *Awaiting Model Update*>,
 (0, 8): <gurobi.Var *Awaiting Model Update*>,
 (0, 9): <gurobi.Var *Awaiting Model Update*>,
 (0, 10): <gurobi.Var *Awaiting Model Update*>,
 (0, 11): <gurobi.Var *Awaiting Model Update*>,
 (0, 12): <gurobi.Var *Awaiting Model Update*>,
 (0, 13): <gurobi.Var *Awaiting Model Update*>,
 (0, 14): <gurobi.Var *Awaiting Model Update*>,
 (0, 15): <gurobi.Var *Awaiting Model Update*>,
 (0, 16): <gurobi.Var *Awaiting Model Update*>,
 (0, 17): <gurobi.Var *Awaiting Model Update*>,
 (0, 18): <gurobi.Var *Awaiting Model Update*>,
 (0, 19): <gurobi.Var *Awaiting Model Update*>,
 (0, 20): <gurobi.Var *Awaiting Model Update*>,
 (

In [351]:
m.addConstrs((y[0, p] == 0 for p in range(i)))
m.addConstrs(gp.quicksum(v[p]*y[m, p] for p in range(i)) <= volume for m in range(t))
m.addConstrs(y[m, p] == x[m, p] - d[m, p] + y[m - 1, p] for p in range(i) for m in range(1, t))
print('hi')

hi


In [352]:
one = gp.quicksum(c[m, p]*x[m, p] for p in range(i) for m in range(t))
two = gp.quicksum((s[p]*y[m, p] for p in range(i) for m in range(t)))

m.setObjective(one + two, GRB.MINIMIZE)
m.optimize()

Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[x86])

CPU model: Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1020 rows, 2000 columns and 3900 nonzeros
Model fingerprint: 0x1de6e6ac
Coefficient statistics:
  Matrix range     [6e-03, 1e+00]
  Objective range  [6e-03, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 1e+02]
Presolve removed 151 rows and 245 columns
Presolve time: 0.01s
Presolved: 869 rows, 1755 columns, 3455 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.7480813e+04   7.681867e+03   0.000000e+00      0s
    1340    9.6126365e+05   0.000000e+00   0.000000e+00      0s

Solved in 1340 iterations and 0.02 seconds (0.01 work units)
Optimal objective  9.612636485e+05


In [358]:
x = []
y = []

status_code = {1:'LOADED', 2:'OPTIMAL', 3:'INFEASIBLE', 4:'INF_OR_UNBD', 5:'UNBOUNDED'}
status = m.status
print('Optimization status is {}'.format(status_code[status]))
if status == 2:
    print('Optimal solution: ')
    for var in m.getVars():
        if var.varName[0] == 'x':
            x.append(var.x)
        else:
            y.append(var.x)
    print('optimal values of x')
    # Rows are months and columns are products
    x = np.array(x)
    x.shape = (t, i)
    x = pd.DataFrame(x)
    print(x)
    print('optimal values of y')
    # Rows are months and columns are products
    y = np.array(y)
    y.shape = (t, i)
    y = pd.DataFrame(y)
    print(y)
    print('Optimal objective value: \n${}'.format(round(m.objVal, 2)))

Optimization status is OPTIMAL
Optimal solution: 
optimal values of x
       0           1      2      3           4      5      6      7      8   \
0     0.0    0.000000    0.0    0.0    0.000000    0.0    0.0    0.0    0.0   
1    22.0   72.000000   68.0   12.0   73.000000  102.0  105.0  110.0  107.0   
2    89.0   83.000000   92.0   68.0  109.000000   27.0    0.0   71.0   23.0   
3   110.0   80.000000   39.0   24.0   91.000000   91.0   89.0  104.0   79.0   
4   112.0  250.833165   77.0  107.0   86.000000   86.0   51.0   58.0   42.0   
5    46.0    0.000000   71.0   19.0   72.000000   81.0    0.0   95.0   76.0   
6    45.0   57.166835   59.0   73.0   69.000000   36.0   33.0   66.0  115.0   
7    90.0  114.000000   71.0   61.0   19.000000   78.0  165.0  114.0  110.0   
8    89.0   90.000000   20.0   47.0  112.517491  111.0    0.0   30.0   70.0   
9    76.0  148.601576  113.0   38.0   53.482509   89.0   45.0  111.0   72.0   
10  108.0   46.398424   72.0   34.0   24.000000  103.0   32.0